In [2]:
import googlemaps
import pandas as pd
import numpy as np
import math
from haversine import haversine

# google maps API Key
gmaps_key = "*****"
gmaps = googlemaps.Client(key=gmaps_key)

# 사용자의 수 및 주소 입력
num = int(input("사람 수: "))
ques = input("가중치를 부여하시겠습니까?(네/아니오)")
weight = []
person = []
if ques == "네":
    for i in range(num):
        person.append(input(f'{i+1}번 사용자 출발지:'))
        weight.append(int(input(f"{i+1}번 가중치:")))
        print(person[i],weight[i])
    print('-----------------------------------------------')
    lat_sum = 0
    lng_sum = 0
    for i in range(num):
        start_info = gmaps.geocode(person[i], language='ko')
        start_geo = start_info[0].get('geometry')
        start_lat = start_geo['location']['lat']
        start_lng = start_geo['location']['lng']
        lat_sum += start_lat * weight[i]
        lng_sum += start_lng * weight[i]
        print(f'{i+1}번 사용자', person[i], "위도: ", start_lat, "경도: ", start_lng)
        print('-----------------------------------------------')

    lat_avg = lat_sum/sum(weight)
    lng_avg = lng_sum/sum(weight)
    loc_avg = (lat_avg, lng_avg)
    print('평균 위치:',loc_avg) # 사용자들의 평균 위도&경도
    print('-----------------------------------------------')
elif ques =="아니오":    
    for i in range(num):
        person.append(input(f'{i+1}번 사용자:'))
    print('-----------------------------------------------')
    lat_sum = 0
    lng_sum = 0
    for i in range(num):
        start_info = gmaps.geocode(person[i], language='ko')
        start_geo = start_info[0].get('geometry')
        start_lat = start_geo['location']['lat']
        start_lng = start_geo['location']['lng']
        lat_sum += start_lat
        lng_sum += start_lng
        print(f'{i+1}번 사용자', person[i], "위도: ", start_lat, "경도: ", start_lng)
        print('-----------------------------------------------')

    lat_avg = lat_sum/num
    lng_avg = lng_sum/num
    loc_avg = (lat_avg, lng_avg)
    print('평균 위치:',loc_avg) # 사용자들의 평균 위도&경도
    print('-----------------------------------------------')
else:
    print("네/아니오 중에서 대답해주세요.")

# 근접도 계산
reweight = [] # 계산된 근접도
proximity = [] # (총합 10이되는) 근접도
dest = []
distance = []
p = []
d = []
distance_sum = 0
reweight_sum = 0

dest.append(input("목적지: "))
dest_info = gmaps.geocode(dest[0], language='ko')
dest_geo = dest_info[0].get('geometry')
dest_lat = dest_geo['location']['lat']
dest_lng = dest_geo['location']['lng']

if dest[0] in person:
    for i in range(num):
        if person[i] != dest[0]:
            start_info = gmaps.geocode(person[i], language='ko')
            start_geo = start_info[0].get('geometry')
            start_lat = start_geo['location']['lat']
            start_lng = start_geo['location']['lng']
            distance.append(math.sqrt((dest_lat-start_lat)**2+(dest_lng-start_lng)**2))
        else:            
            distance.append(int("0"))
else:
    for i in range(num):
        start_info = gmaps.geocode(person[i], language='ko')
        start_geo = start_info[0].get('geometry')
        start_lat = start_geo['location']['lat']
        start_lng = start_geo['location']['lng']
        distance.append(math.sqrt((dest_lat-start_lat)**2+(dest_lng-start_lng)**2))
distance_sum += distance[i]
        
for j in range(num):
    try:
        reweight.append(float(distance_sum/distance[j])) # 근접도 계산 및 입력    
        reweight_sum += reweight[j]
    except ZeroDivisionError:
        reweight.append(int("0"))
        p.append(int("0"))
    
for k in range(num):
    proximity.append(reweight[k]*10/reweight_sum) #근접도 총합 10 되도록 보정
    p = round(proximity[k], 1)    
    print(f'{k+1}번 사용자 근접도: ', p)
    print('-----------------------------------------------')            
    
# 비용 정산
cost = int(input("결제비용: "))
devided = []
if dest[0] in person:
    for i in range(num):
        if reweight[i] != 0:
            devided.append(float(cost/2*reweight[i]/sum(reweight)))        
        else:
            devided.append(int(cost/2))
        d = int(round(devided[i], -2))
        print(f'{i+1}번 사용자', person[i], "부담금: ",d,"원")
else:
    for i in range(num):
        devided.append(float(cost*reweight[i]/sum(reweight)))        
        d = int(round(devided[i], -2))
        print(f'{i+1}번 사용자', person[i], "부담금: ",d,"원")
print('-----------------------------------------------')
        
# 지하철역 출력
station_name = [] # 지하철역 집합 리스트
station = pd.read_csv('subway.csv', encoding = 'euc-kr')['역이름']
for name in station:
    station_name.append(name + '역')

#지하철역과 사용자의 평균 위치의 차이 출력
k = 0
distance = 0
dic_all = [] # 거리 차이의 집합 리스트
for i in station_name[0:30]:
    try:
        station_info = gmaps.geocode(f'{i}', language='ko')
        station_geo = station_info[0].get('geometry')
        station_lat = station_geo['location']['lat'] # 위도
        station_lng = station_geo['location']['lng'] # 경도
        station_loc = (station_lat, station_lng) # 위도, 경도를 담은 튜플
        distance = haversine(loc_avg, station_loc) # 사용자들의 평균 위치와 역 간의 거리  
        dic_all.append(distance)
        k += 1
        # print(k, "번", i, "위도:", station_lat, "경도:", station_lng, "역과의 거리:", distance) 
    except IndexError :
        if i == '동대문역':
            station_lat = 37.571697
            station_lng = 127.01086
            station_loc = (station_lat, station_lng) # 위도, 경도를 담은 튜플
            distance = haversine(loc_avg, station_loc) # 사용자들의 평균 위치와 역 간의 거리  
            dic_all.append(distance)
            k += 1
            # print(k, "번", i, "위도:", station_lat, "경도:", station_lng, "역과의 거리:", distance) 
        elif i == '삼양역':
            station_lat = 37.626985
            station_lng = 127.018141
            station_loc = (station_lat, station_lng) # 위도, 경도를 담은 튜플
            distance = haversine(loc_avg, station_loc) # 사용자들의 평균 위치와 역 간의 거리  
            dic_all.append(distance)
            k += 1
            # print(k, "번", i, "위도:", station_lat, "경도:", station_lng, "역과의 거리:", distance)    
        else: # 4.19 묘지역
            station_lat = 37.64942
            station_lng = 127.013708
            station_loc = (station_lat, station_lng) # 위도, 경도를 담은 튜플
            distance = haversine(loc_avg, station_loc) # 사용자들의 평균 위치와 역 간의 거리  
            dic_all.append(distance)
            k += 1
            # print(k, "번", i, "위도:", station_lat, "경도:", station_lng, "역과의 거리:", distance)  

사람 수: 3
가중치를 부여하시겠습니까?(네/아니오)아니오
1번 사용자:교대역
2번 사용자:역삼역
3번 사용자:선릉역
-----------------------------------------------
1번 사용자 교대역 위도:  37.4934705 경도:  127.0142285
-----------------------------------------------
2번 사용자 역삼역 위도:  37.4999072 경도:  127.0373932
-----------------------------------------------
3번 사용자 선릉역 위도:  37.504479 경도:  127.048941
-----------------------------------------------
평균 위치: (37.499285566666664, 127.0335209)
-----------------------------------------------
목적지: 강남역
0.022299658495150313
1번 사용자 근접도:  3.3
-----------------------------------------------
2번 사용자 근접도:  4.6
-----------------------------------------------
3번 사용자 근접도:  2.1
-----------------------------------------------
결제비용: 120000
1번 사용자 교대역 부담금:  39300 원
2번 사용자 역삼역 부담금:  55700 원
3번 사용자 선릉역 부담금:  24900 원
-----------------------------------------------


In [25]:
import operator
import folium

x = dict(zip(station_name, dic_all))
sort = sorted(x.items(), key=lambda t : t[1])
min_loc = sort[0:11]
min_loc
fm8 = folium.Map(location = [37.748362, 127.044251])
for i in range(len(min_loc)):
    min_info = gmaps.geocode(min_loc[i][0], language='ko')
    min_geo = min_info[0].get('geometry')
    min_lat = min_geo['location']['lat'] # 위도
    min_lng = min_geo['location']['lng'] # 경도
    folium.Marker([min_lat, min_lng], popup=min_loc[i][0]).add_to(fm8)
    
for i in range(num):
    station_info = gmaps.geocode(person[i], language='ko')
    station_geo = station_info[0].get('geometry')
    station_lat = station_geo['location']['lat']
    station_lng = station_geo['location']['lng']
    folium.CircleMarker([station_lat, station_lng], popup=person[i], radius=15, color='#3186cc', fill_color= '#3186cc').add_to(fm8)
    
folium.CircleMarker(loc_avg, popup=person[i], radius=60, color='#ff0000', fill_color= '#ff0000').add_to(fm8)
fm8